In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
spark = SparkSession.builder.appName("Python Spark DataFrames basic example").config("spark.some.config.option", "some-value").getOrCreate()

df = spark.read.csv("./music_streaming.csv", header=True)

In [ ]:
# 1. Remove duplicates (Artist Name + Track Name)
df = df.dropDuplicates(subset=['Artist Name', 'Track Name'])

In [ ]:
# 2. Missing values in each column
df.select([col(c).isNull().alias(c) for c in df.columns]).show()

In [ ]:
# Remove rows with missing values in the 'Popularity' column
df = df.dropna(subset=['Popularity'])

In [ ]:
# Drop unnecessary column 'key'
df = df.drop('key')

In [ ]:
# Impute missing values in 'instrumentalness' column with mean value per artist
df = df.withColumn('instrumentalness', col('instrumentalness').cast('double'))
mean_instrumentalness = df.groupBy('Artist Name').agg({'instrumentalness': 'mean'})
df = df.join(mean_instrumentalness, on='Artist Name', how='left')
df = df.withColumn('instrumentalness', col('instrumentalness').fillna(mean_instrumentalness['avg(instrumentalness)']))

In [ ]:
# Plot boxplot of 'loudness'
df.select('loudness').toPandas().boxplot()

In [ ]:
# Remove outliers using IQR method for 'loudness'
Q1 = df.approxQuantile('loudness', [0.25], 0.05)[0]
Q3 = df.approxQuantile('loudness', [0.75], 0.05)[0]
IQR = Q3 - Q1
df = df.filter((col('loudness') >= Q1 - 1.5 * IQR) & (col('loudness') <= Q3 + 1.5 * IQR))

In [ ]:
# Convert 'duration_in min/ms' to minutes if greater than 1000, assuming milliseconds
df = df.withColumn('duration_in min/ms', col('duration_in min/ms').cast('double'))
df = df.withColumn('duration_in min/ms', col('duration_in min/ms') / 60000).where(col('duration_in min/ms') <= 29.8)

df.show()

In [ ]:
# Display songs with duration greater than 5 minutes
df_filtered = df.filter(col('duration_in min/ms') > 5)
df_filtered.show()

In [ ]:
# Display songs by J. Cole, Novo Amor, and Anson Seabra
df_filtered.filter(col('Artist Name').isin('J. Cole', 'Novo Amor', 'Anson Seabra')).show()

In [ ]:
# Count the number of songs in each genre
df.groupBy('Genre').count().show()

In [ ]:
# Count the number of songs for each artist
df.groupBy('Artist Name').count().sort(col('count').desc()).show()

In [ ]:
# Mean popularity for each artist
df.groupBy('Artist Name').avg('Popularity').sort(col('avg(Popularity)').desc()).show()

In [ ]:
# Top 10 songs based on popularity
df.orderBy(col('Popularity').desc()).select('Track Name', 'Artist Name', 'Popularity').limit(10).show()

In [ ]:
# Recommend songs for a party based on danceability, tempo, loudness, and valence
df.orderBy(col('danceability').desc(), col('tempo').desc(), col('loudness').desc(), col('valence').desc()).select('Track Name', 'Artist Name', 'danceability', 'tempo', 'loudness', 'valence').limit(5).show()